In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import plotly.express as px
import cufflinks as cf

In [2]:
# Load financial time series data from a CSV file
data = pd.read_csv('AAPL_2006-01-01_to_2018-01-01.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

In [3]:
# Select the feature(s) you want to use for prediction
features = ['Open', 'High', 'Low', 'Close']
data = data[features]

In [4]:
# Prepare the dataset
#prices = data['Close'].values.reshape(-1, 1)

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

In [5]:
# Define the sequence length and split the data into sequences
sequence_length = 10
sequences = []
next_value = []
for i in range(len(scaled_data) - sequence_length):
    sequences.append(scaled_data[i:i+sequence_length])
    next_value.append(scaled_data[i+sequence_length])

sequences = np.array(sequences)
next_value = np.array(next_value)

In [6]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(sequences, next_value, test_size=0.2, random_state=42)


In [7]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(sequence_length, len(features))))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(len(features)))
model.compile(optimizer='adam', loss='mean_squared_error')

In [8]:
# Train the model
model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
38/38 [==============================] - 1s 13ms/step - loss: 0.0312 - val_loss: 0.0018
Epoch 2/50
38/38 [==============================] - 0s 6ms/step - loss: 6.6394e-04 - val_loss: 3.0589e-04
Epoch 3/50
38/38 [==============================] - 0s 6ms/step - loss: 2.9422e-04 - val_loss: 2.5832e-04
Epoch 4/50
38/38 [==============================] - 0s 6ms/step - loss: 2.7185e-04 - val_loss: 2.8203e-04
Epoch 5/50
38/38 [==============================] - 0s 6ms/step - loss: 2.6352e-04 - val_loss: 2.4589e-04
Epoch 6/50
38/38 [==============================] - 0s 6ms/step - loss: 2.6020e-04 - val_loss: 2.4993e-04
Epoch 7/50
38/38 [==============================] - 0s 6ms/step - loss: 2.5771e-04 - val_loss: 2.4117e-04
Epoch 8/50
38/38 [==============================] - 0s 6ms/step - loss: 2.6015e-04 - val_loss: 2.3709e-04
Epoch 9/50
38/38 [==============================] - 0s 6ms/step - loss: 2.6477e-04 - val_loss: 2.4324e-04
Epoch 10/50
38/38 [==============================] - 

In [9]:
# Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

19/19 [==============================] - 0s 1ms/step


In [10]:
# Convert predictions and y_test to DataFrames
predictions_df = pd.DataFrame(predictions, columns=features)
y_test_df = pd.DataFrame(y_test, columns=features)

In [11]:
# Plot the results using Cufflinks
cf.go_offline()
predictions_df.iplot(title='Predictions vs. Actual', xTitle='Index', yTitle='Price')
y_test_df.iplot(title='Actual Prices', xTitle='Index', yTitle='Price')